# Import

In [393]:
import numpy as np
import pandas as pd
import csv
import datetime
import math
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

# My Library

In [394]:
#製造answer.csv的第1列
def answer_header():
    header = []
    for i in range(28):
        header.append('time_slot_' + str(i) )
    return header

# Read File

In [395]:
reference = pd.read_csv('reference8.csv')
answer_v2 = pd.read_csv('answer_v28.csv')
the_labels = pd.read_csv('public/The_Labels.csv')

# Main

In [396]:
params = {'boosting_type': 'gbdt',
          'max_depth' : -1,
          'objective': 'binary',
          'nthread': 8,
          'num_leaves': 37,
          'learning_rate': 0.0479,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 0.889,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 0.14046,
          'reg_lambda': 1.20195,
          'min_split_gain': 0.5,
          'min_child_weight': 0.9998,
          'min_child_samples': 5,
          'scale_pos_weight': 1.30022,
          'num_class' : 1,
          'metric' : 'auc',
         }
# 目標調整參數(更改以下7個位置)
scale_pos_weights = [1.30015,1.30016,1.30017,1.30018,1.30019,1.3002,1.30021,1.30022,1.30023,1.30024,1.30025]

# 讀取資料、切資料
X = reference[answer_header()]
Y = the_labels[answer_header()]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=33)

# 初始化
y = []
model = []
the_summary = 0
the_result = 0

# 依序套用參數
for scale_pos_weight in scale_pos_weights:
    params['scale_pos_weight'] = scale_pos_weight
    
    # 使用目前參數訓練模型
    for i in range(0,28):
        y.append(y_train['time_slot_' + str(i)])
        model.append(lgb.train(params, lgb.Dataset(X_train, y[i])))
        print(i)
    
    # 初始化
    summ = 0
    y = []
    
    # 計算目前參數之得分
    for i in range(0,28):
        y.append(y_test['time_slot_' + str(i)])
        score = metrics.roc_auc_score(y[i], model[i].predict(X_test))
        summ += score
        print(i, ': ', score)
    print(scale_pos_weight, ' average: ', summ/28)
    
    # 更新最佳參數
    if summ > the_summary:
        the_summary = summ
        the_result = scale_pos_weight
        print('update: ', the_result)
    
    # 重新初始化
    y = []
    model = []

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0 :  0.8699858672839991
1 :  0.8662557810304974
2 :  0.893763138137887
3 :  0.9030201509805293
4 :  0.8773063123541645
5 :  0.8550671968447491
6 :  0.8872090223951214
7 :  0.8914354433873533
8 :  0.8673548496650427
9 :  0.8612010198853333
10 :  0.8820901061864681
11 :  0.8960068324483922
12 :  0.8639132051488961
13 :  0.8436557248904539
14 :  0.8695858708485542
15 :  0.8796662550106071
16 :  0.8531096653158108
17 :  0.8287186401008707
18 :  0.8511547441300997
19 :  0.8517799518423654
20 :  0.8210042041987778
21 :  0.8233175595233022
22 :  0.8391662527001063
23 :  0.8420561709260584
24 :  0.8096932153314998
25 :  0.8182812824158279
26 :  0.8527964416897456
27 :  0.8642602722236186
1.30015  average:  0.8593876848891476
update:  1.30015
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
0 :  0.8699858481105952
1 :  0.8662558507276523
2 :  0.893763138137887
3 :  0.9030201509805293
4 :  0.877306

27
0 :  0.869985733070172
1 :  0.8662558367882214
2 :  0.8937634056773387
3 :  0.9031939000489957
4 :  0.8773068797582668
5 :  0.8555639019190521
6 :  0.8872090654305513
7 :  0.8914352361732782
8 :  0.8673549784469654
9 :  0.8616916779194783
10 :  0.8820907213195412
11 :  0.8960070392103349
12 :  0.8639131294056643
13 :  0.8436556716060428
14 :  0.8695862312483962
15 :  0.8796235107825958
16 :  0.8531095703509757
17 :  0.8286679223220895
18 :  0.8511547822663317
19 :  0.8517797916977058
20 :  0.8210041870873932
21 :  0.8233252880858419
22 :  0.8396657909968612
23 :  0.8420561709260583
24 :  0.8096930913393084
25 :  0.8182815701568793
26 :  0.8527964875144968
27 :  0.8644487779215708
1.30025  average:  0.8594507267668002


In [397]:
the_result

1.30022

# Check File

In [398]:
reference

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.007812,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000977,0.000000,0.000000,0.000977,0.000000,0.000000,0.000000
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.125000,0.500000,0.000000
3,3,0.000000,0.000488,0.000000,0.000000,0.000488,0.500488,0.000000,0.000000,0.500000,...,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.500244,0.000244,0.000000
4,4,0.468750,0.968750,0.000000,0.812500,0.968750,0.968750,0.000000,1.562500,0.968750,...,0.000000,0.328125,0.484375,0.484375,0.328125,0.062500,0.234375,0.484375,0.000000,0.546875
5,5,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000
6,6,0.000488,0.691284,0.000488,0.000610,0.063110,0.507690,0.000244,0.000000,0.000000,...,0.001099,0.125244,0.948608,0.572144,0.500000,1.071533,0.751831,0.813904,0.000000,0.000000
7,7,0.625000,0.562500,0.562500,0.000000,0.687500,0.625000,0.500000,0.000000,0.687500,...,0.625000,0.000000,0.656250,0.812500,0.812500,0.500000,0.500000,0.937500,0.812500,0.000000
8,8,0.000977,0.001953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000


In [399]:
answer_v2[answer_header()]

,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,time_slot_9,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,0.843506,0.843140,0.085327,0.151123,0.874878,0.991821,0.234009,0.262939,0.724487,0.734253,...,0.597656,0.164062,0.196289,0.065674,0.003174,0.317627,0.757080,0.921143,0.668945,0.450195
1,0.015625,0.765625,0.015625,0.000000,0.046875,0.796875,0.015625,0.000000,0.671875,0.937988,...,0.023438,0.000000,0.000000,0.156250,0.015625,0.000000,0.000000,0.148438,0.257812,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.125000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,...,0.522461,0.000000,0.000977,0.008545,0.026123,0.000000,0.000244,0.125244,0.125000,0.000000
4,0.250000,0.250000,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.250000,0.000000,0.000000,0.000000,0.125000,0.000000,0.125000,0.000000,0.000000
6,0.500000,0.581055,0.125977,0.001953,0.000000,0.961914,0.382812,0.330078,0.000000,0.331055,...,0.000977,0.000000,0.313477,0.070312,0.250000,0.003906,0.254395,0.383301,0.250000,0.258301
7,0.000000,0.750000,0.000000,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,0.750000,...,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.125000,0.375000,0.250000,0.000000
8,0.516479,0.894409,0.014648,0.000000,0.881714,0.929565,0.005859,0.000000,0.799683,0.928589,...,0.503052,0.000000,0.268555,0.766846,0.018066,0.000488,0.777527,0.887756,0.004944,0.000000
9,0.863159,0.991577,0.000000,0.961060,0.991699,0.929077,0.000000,1.124878,0.874878,0.935425,...,0.000000,0.004883,0.208496,0.014648,0.000000,0.507874,0.090881,0.000061,0.000000,1.300171


In [400]:
the_labels

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,1,0,0,0,1,0,0,0,...,1,1,1,1,1,0,1,0,0,0
7,7,1,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
